In [1]:
import sys
sys.path.append('../')

import numpy as np

from ForwardModeling.ForwardProcessing1D import forward_with_trace_calcing
from Inversion.Strategies.SeismDiffInversion1D import inverse
from Inversion.Optimizators.Optimizations import LBFGSBOptimization, DifferentialEvolution, TrustKrylov, TrustConstr, \
    ConjugateGradient
from Tests.test_ForwardProcessing1D import get_model_2layered, get_model_3layered
from Data.geol_models import get_model_3layered_fluid_rp_dicts
from Objects.Data.WavePlaceholder import OWT, WaveDataPlaceholder
from Objects.Models.Models import SeismicModel1D
from Objects.Attributes.RockPhysics.RockPhysicsAttribute import RockPhysicsAttribute
from Objects.Models.Layer1D import Layer1D, LayerOPT
import time

from matplotlib import pyplot as plt
from Inversion.Strategies.SeismDiffInversion1D import func_to_optimize

In [2]:
def plot_1D_err(forward_params, nvals, min_max=None, dot=False, vline_x=None):
    
    if min_max is None:
        min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
        max_val = forward_params['model'].get_optimization_option("max", vectorize=True)
        
    else:
        min_val = np.array([min_max[0]])
        max_val = np.array([min_max[1]])
    
    dval = (max_val - min_val) / nvals

    val_x = [min_val + dval*i for i in range(nvals+1)]
    errs = []
    for val in val_x:
        errs.append(func_to_optimize(val, placeholders, forward_params, helper=None, show_tol=False))
        
    plt.plot(val_x, errs)
    if dot:
        plt.plot(val_x, errs, 'o')
        
    if vline_x is not None:
        plt.vlines(vline_x, ymin=min(errs), ymax=max(errs), colors='r')
        
    plt.show()

def optimization_func_2D(forward_params, x, y):
    indexes_1 = np.arange(x.shape[0])
    indexes_2 = np.arange(x.shape[1])
    
    Z = np.zeros((x.shape[0], x.shape[1]))
    
    for ind1 in indexes_1:
        for ind2 in indexes_2:
            Z[ind1, ind2] = func_to_optimize([x[ind1, ind2], y[ind1, ind2]], placeholders, forward_params, helper=None, show_tol=False)
        
    return Z

def plot_2D_err(forward_params, nvals, ncontous=10, points=None):
    min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
    max_val = forward_params['model'].get_optimization_option("max", vectorize=True)
    
    dval = (max_val - min_val) / nvals

    val_1_x = [min_val[0] + dval[0]*i for i in range(nvals[0]+1)]
    val_2_x = [min_val[1] + dval[1]*i for i in range(nvals[1]+1)]

    X, Y = np.meshgrid(val_1_x, val_2_x)

    Z = optimization_func_2D(forward_params, X, Y)
    
    plt.contourf(X, Y, Z, ncontous, cmap='seismic')
    plt.colorbar()
    
    if points is not None:
        plt.plot(points[0], points[1], marker='x', color='r')

# 2-layered model

In [20]:
layer_1 = Layer1D(h,
                  rp_attribute=RockPhysicsAttribute(layer_1_dict["components"], layer_1_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

layer_2 = Layer1D(-1,
                  rp_attribute=RockPhysicsAttribute(layer_2_dict["components"], layer_2_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

NameError: name 'h' is not defined

In [ ]:
dx = 100
nx = 20
x_rec = [i * dx for i in range(1, nx)]
wave_types = [OWT.PdPu]

In [ ]:
model = SeismicModel1D([layer_1, layer_2])
model_true = SeismicModel1D([layer_1, layer_2])
model_opt = SeismicModel1D([layer_1, layer_2])

In [ ]:
observe, test_seismic = \
    forward_with_trace_calcing(model, x_rec,
                               dt=3e-03, trace_len=1500, wavetypes=wave_types, display_stat=True,
        visualize_res=False)

In [ ]:
forward_params = {
    "model": model,
    "x_rec": x_rec,
    "dt": 3e-03,
    "trace_len": 1500,
    "wavetypes": wave_types,
    "display_stat": False,
    "visualize_res": False
}

In [ ]:
placeholders = {}
for wt in wave_types:
    placeholders[wt] = WaveDataPlaceholder(
        wt,
        test_seismic[wt]["rays"],
        test_seismic[wt]["seismogram"]
    )

In [ ]:
optimizers = [
    LBFGSBOptimization(
        maxiter=15000,
        maxfun=15000,
        factr=10000,
        maxls=50,
        epsilon=0.000001
    )
]

In [ ]:
optimizers = [
    DifferentialEvolution()
]

In [ ]:
forward_params['model'].layers[0]['Km'] = 10

In [ ]:
forward_params['model'].layers[1]['Km'] = 25

In [ ]:
forward_params['model'].layers[0].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
forward_params['model'].layers[1].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
inversed_model = inverse(optimizers, error=0.01, placeholders=placeholders, forward_params=forward_params)

In [ ]:
inversed_model

## Single variable opt

In [ ]:
from Inversion.Strategies.SeismDiffInversion1D import func_to_optimize

In [ ]:
forward_params['model'].layers[0].rp_attribute.vals_dict['Km']["optimize"] = False

In [ ]:
forward_params['model'].layers[1].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
forward_params['model'].layers[0]['Km'] = 7.3

In [ ]:
forward_params['model'].layers[1]['Km'] = 21.5

In [ ]:
forward_params['model'].get_optimization_option("val", vectorize=True)

In [ ]:
min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
max_val = forward_params['model'].get_optimization_option("max", vectorize=True)

In [ ]:
nvals = 30

In [ ]:
dval = (max_val - min_val) / nvals

val_x = [min_val + dval*i for i in range(nvals+1)]
errs = []
for val in val_x:
    errs.append(func_to_optimize(val, placeholders, forward_params, helper=None, show_tol=False))

In [ ]:
plt.plot(val_x, errs)

## Duo variable opt

In [ ]:
forward_params['model'].layers[1].rp_attribute.vals_dict['Km']["optimize"] = True

In [ ]:
forward_params['model'].get_optimization_option("val", vectorize=True)

In [ ]:
min_val = forward_params['model'].get_optimization_option("min", vectorize=True)
max_val = forward_params['model'].get_optimization_option("max", vectorize=True)

In [ ]:
def f(x, y):
    return np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)


In [ ]:
dval = (max_val - min_val) / nvals

val_1_x = [min_val[0] + dval[0]*i for i in range(nvals[0]+1)]
val_2_x = [min_val[1] + dval[1]*i for i in range(nvals[1]+1)]

X, Y = np.meshgrid(val_1_x, val_2_x)

Z = optimization_func_2D(X, Y)

In [ ]:
Z.shape

In [ ]:
X.shape

In [ ]:
plt.contourf(X, Y, Z, 10, cmap='cool')
plt.colorbar()

# 3-layered model

In [3]:
h = 1000

In [4]:
layer_1_dict, layer_2_dict, layer_3_dict = get_model_3layered_fluid_rp_dicts()

In [5]:
layer_3_1 = Layer1D(h,
                  rp_attribute=RockPhysicsAttribute(layer_1_dict["components"], layer_1_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

layer_3_2 = Layer1D(h,
                  rp_attribute=RockPhysicsAttribute(layer_2_dict["components"], layer_2_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

layer_3_3= Layer1D(-1,
                  rp_attribute=RockPhysicsAttribute(layer_3_dict["components"], layer_2_dict["name"]),
                  seism_attribute=None,
                  opt=LayerOPT.RP)

In [6]:
dx = 300
nx = 5
x_rec = [i * dx for i in range(1, nx+1)]
# wave_types = [OWT.PdPu]
wave_types = [OWT.PdPu, OWT.PdSVu]

In [7]:
model_3l = SeismicModel1D([layer_3_1, layer_3_2, layer_3_3])

In [8]:
observe, test_seismic = \
    forward_with_trace_calcing(model_3l, x_rec,
                               dt=3e-03, trace_len=1500, wavetypes=wave_types, display_stat=True,
        visualize_res=False)

Calculating rockphysics model...
Rockphysics model calculated!
Calculating PdPu-rays...
Calculating PdPu-reflections...
Calculating p-refractions...
Calculating PdSVu-rays...
Calculating PdSVu-reflections...
Calculating p-refractions...


In [9]:
forward_params = {
    "model": model_3l,
    "x_rec": x_rec,
    "dt": 3e-03,
    "trace_len": 1500,
    "wavetypes": wave_types,
    "display_stat": False,
    "visualize_res": False
}

In [10]:
placeholders = {}
for wt in wave_types:
    placeholders[wt] = WaveDataPlaceholder(
        wt,
        test_seismic[wt]["rays"],
        test_seismic[wt]["seismogram"]
    )

In [11]:
optimizers_lbfgs = [
    LBFGSBOptimization(
        maxiter=15000,
        maxfun=15000,
#         factr=10000,
#         maxls=50,
#         epsilon=0.000001
    )
]

In [12]:
optimizers_cg = [
    ConjugateGradient()
]

In [13]:
optimizers_de = [
    DifferentialEvolution(
          popsize= 5,
          maxiter= 20000,
          init= "latinhypercube",
          strategy= "best1bin",
          disp= False,
          polish= False,
          tol= 0.00001,
          mutation= 1.5,
          recombination= 0.6,
          workers= 8
    )
]

In [14]:
optimizers_tc = [
    TrustConstr(
        opt_options={
            "initial_tr_radius": 0.7,
            "maxiter": 1000000
#             "gtol":1e-4,
        }
    ),
#     LBFGSBOptimization(
#         maxiter=15000,
#         maxfun=15000,
#         factr=10000,
#         maxls=50,
#         epsilon=0.000001
#     )
]

### Some model changes

In [ ]:
forward_params['model'].layers[0].rp_attribute.vals_dict['phi_s']['optimize'] = False
forward_params['model'].layers[0].rp_attribute.vals_dict['Kf']['optimize'] = False
forward_params['model'].layers[0].rp_attribute.vals_dict['rho_f']['optimize'] = False
forward_params['model'].layers[0].rp_attribute.vals_dict['phi']['optimize'] = False

forward_params['model'].layers[1].rp_attribute.vals_dict['phi_s']['optimize'] = False
forward_params['model'].layers[1].rp_attribute.vals_dict['Kf']['optimize'] = False
forward_params['model'].layers[1].rp_attribute.vals_dict['rho_f']['optimize'] = False
forward_params['model'].layers[1].rp_attribute.vals_dict['phi']['optimize'] = False

### Km optimization

In [15]:
opt_flag = True
# opt_flag = False
if opt_flag:
    forward_params['model'].layers[0]["Km"] = 9
    forward_params['model'].layers[0].opt = LayerOPT.RP
    forward_params['model'].layers[0].rp_attribute.vals_dict['Km']['optimize'] = True
    
else:
    forward_params['model'].layers[0]["Km"] = 7.3
    forward_params['model'].layers[0].opt = LayerOPT.NO
    forward_params['model'].layers[0].rp_attribute.vals_dict['Km']['optimize'] = False

opt_flag = True
# opt_flag = False

if opt_flag:
    forward_params['model'].layers[1]["Km"] = 25
    forward_params['model'].layers[1].opt = LayerOPT.RP
    forward_params['model'].layers[1].rp_attribute.vals_dict['Km']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[1]["Km"] = 21.5
    forward_params['model'].layers[1].opt = LayerOPT.NO
    forward_params['model'].layers[1].rp_attribute.vals_dict['Km']['optimize'] = False



# opt_flag = True
opt_flag = False

if opt_flag:
    forward_params['model'].layers[2]["Km"] = 25
    forward_params['model'].layers[2].opt = LayerOPT.RP
    forward_params['model'].layers[2].rp_attribute.vals_dict['Km']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[2]["Km"] = 22
    forward_params['model'].layers[2].opt = LayerOPT.NO
    forward_params['model'].layers[2].rp_attribute.vals_dict['Km']['optimize'] = False



### Gm optimization

In [21]:
# opt_flag = True
opt_flag = False
if opt_flag:
    forward_params['model'].layers[0]["Gm"] = 5
    forward_params['model'].layers[0].opt = LayerOPT.RP
    forward_params['model'].layers[0].rp_attribute.vals_dict['Gm']['optimize'] = True
    
else:
    forward_params['model'].layers[0]["Gm"] = 2.71
    forward_params['model'].layers[0].opt = LayerOPT.NO
    forward_params['model'].layers[0].rp_attribute.vals_dict['Gm']['optimize'] = False

# opt_flag = True
opt_flag = False

if opt_flag:
    forward_params['model'].layers[1]["Gm"] = 15
    forward_params['model'].layers[1].opt = LayerOPT.RP
    forward_params['model'].layers[1].rp_attribute.vals_dict['Gm']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[1]["Gm"] = 17.5
    forward_params['model'].layers[1].opt = LayerOPT.NO
    forward_params['model'].layers[1].rp_attribute.vals_dict['Gm']['optimize'] = False



# opt_flag = True
opt_flag = False

if opt_flag:
    forward_params['model'].layers[2]["Gm"] = 15
    forward_params['model'].layers[2].opt = LayerOPT.RP
    forward_params['model'].layers[2].rp_attribute.vals_dict['Gm']['optimize'] = True

else:
    # TRUE VALUE
    forward_params['model'].layers[2]["Gm"] = 10.7
    forward_params['model'].layers[2].opt = LayerOPT.NO
    forward_params['model'].layers[2].rp_attribute.vals_dict['Gm']['optimize'] = False



NameError: name 'forward_params' is not defined

## Model calcing

In [16]:
forward_params['model'].get_optimization_option('val')

OrderedDict([(0, {'Km': 9}),
             (1,
              {'Km': 25,
               'phi_s': 0.05,
               'Kf': 1.8,
               'rho_f': 0.95,
               'phi': 0.1})])

In [17]:
# scale="minmax"
scale=None
start_time = time.time()
# optimizers_tc[0].opt_options["finite_diff_rel_step"] = 0.1
inversed_model = inverse(optimizers_tc, error=0.07, placeholders=placeholders, forward_params=forward_params, scale=scale)
print(f"TC time: {time.time() - start_time}")

0.14651545438046212
0.14651545383051245
99999
0.1465154511795146
0.1465154545942669
0.14651545446906183
0.14651543860308203
0.14518164923740112
0.14518164870868291
0.14518165055241936
0.14518164613196727
0.14518164943529346
0.14518164932845443
0.14518163390264624
0.14606303841794188
0.14606303788663846
0.1460630397519434
0.14606303531048415
0.1460630386206438
0.14606303850857208
0.14606302308584007
0.1462217583326162
0.14622175779589217
0.14622175968726842
0.14622175519956385
0.14622175853864308
0.14622175842266488
0.1462217428722753
0.14636669791962187
0.14636669737661762
0.14636669929786073
0.1463666947552395
0.14636669812914152
0.14636669800897312
0.14636668231669864
0.14642237127450644
0.146422370728271
0.14642237266598224
0.14642236809337764
0.14642237148599724
0.14642237136351605
0.1464223555901963
0.14649625449387652
0.14649625394491683
0.14649625589571252
0.14649625129876637
0.14649625470687988
0.14649625458258989
0.14649623874699674
0.14646089091817796
0.14646089036969928
0.14

/home/apenkin/miniconda3/envs/rpsi/lib/python3.7/site-packages/scipy/optimize/_hessian_update_strategy.py:187: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  'approximations.', UserWarning)


0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.1463373364689769
0.14633733190602816
0.14633733529301418
0.1463373351716383
0.14633731942803715
0.14633733508234467
0.14633733453827077
0.146337336468

0.13239412663798417
0.1323941264496025
0.13239412621779484
0.13239412747911364
0.13239412656714486
0.13239412667903946
0.13239412981972992
0.13239412663798417
0.13239412644960252
0.13239412621779484
0.13239412747911364
0.13239412656714486
0.13239412667903944
0.13239412981972992
0.13155981844674555
0.13155981827334035
0.13155981802445751
0.13155981932024033
0.1315598183750773
0.13155981848958645
0.13155982170955438
0.13286033957420304
0.13286033935674454
0.13286033916374976
0.13286034036019065
0.13286033950568946


KeyboardInterrupt: 

In [18]:
# scale="minmax"
scale=None
start_time = time.time()
inversed_model = inverse(optimizers_de, error=0.01, placeholders=placeholders, forward_params=forward_params, scale=scale)
print(f"DE time: {time.time() - start_time}")

/home/apenkin/miniconda3/envs/rpsi/lib/python3.7/site-packages/scipy/optimize/_differentialevolution.py:454: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  " updating='deferred'", UserWarning)


0.14205766581549692
0.15531066652944564
0.19747929449043267
0.11689840863437234
0.13348923273410127
0.14815620727296255
0.13615302382150157
0.14913759989285463
0.15422226905835937
0.17741280928449804
0.15189021791087787
0.1393451922236805
0.13645897966837944
0.2180945522331457
0.13410278943426046
0.12385056836571615
0.17934673895618547
0.19384146470562605
0.19751334988265412
0.1212420855507561
0.15047023266585266
0.132097505352178
0.15209904538424168
0.12944388826794398
0.20478372370051268
0.20868092849889608
0.12195823055057711
0.1706100692474743
0.16467774539842267
0.11032396113178206
0.11080073336886054
0.16750352320692075
0.1463879191564911
0.16006668003916039
0.14966703093646194
0.13557330957497826
0.15018581848821067
0.1361135998523392
0.1537692138871092
0.13619242019156405
0.19208168297043343
0.1531352715132953
0.15271619452921026
0.14168901010754661
0.1283796941099863
0.15735758661204613
0.13522957270116687
0.14081667908218698
0.143285478041259
0.19270157490363482
0.11495345820

0.11178812153249237
0.13942594972419506
0.13190066157327296
0.11414141681704491
0.10298805934563363
0.1348666862648276
0.12122091911429686
0.10971958913012023
0.18573710342083932
0.10228361316044426
0.10513101583095606
0.1882747632559954
0.10723482103956858
0.14768995859604345
0.11275974855820556
0.14631789566972464
0.1631104840865219
0.14396143426005942
0.15191641394673508
0.12307074296703235
0.12410708300479152
0.08583220824737067
0.12586240909833862
0.12131012449880538
0.12021542209648572
0.06547419882013815
0.13354021040989872
0.1508040696446173
0.21345935821088374
0.17233398276839695
0.17593508296400912
0.1552089844857688
0.06863433818067526
0.15309241225266687
0.11053814034760234
0.12243622445103422
0.09311882682339839
0.152706289679449
0.1142385396703671
0.14410225022075362
0.10531662899210428
0.09403996460887798
0.15704332673501264
0.1459154145515668
0.09824754326602533
0.11904365036130851
0.11050611126242527
0.17303631434179817
0.11041433811491107
0.1308792148993558
0.14140569

0.12699392988056285
0.06608541168921739
0.10965577821274733
0.11087968582731256
0.14189899863900224
0.11768022138240689
0.10001295532429909
0.10177111046999517
0.10660110534581162
0.13047291439907938
0.11680178967110309
0.11222884141545036
0.0624208985685787
0.09053393085916459
0.1011657078586137
0.10126711312602452
0.13176689189145868
0.08814719323091566
0.06909147929554027
0.0831994444253122
0.16262094010099462
0.08171735658228187
0.12240904602683234
0.06796882904720528
0.09751324913124998
0.11512979407424269
0.14700825625429165
0.08770657238120354
0.14292394884388498
0.09738128281572625
0.11427068382313404
0.1480355951278362
0.1169923410826113
0.107151398959003
0.14048497608389757
0.09745563925934936
0.08676581286319693
0.1274241541625548
0.07551813627197676
0.1297347633599055
0.14249787991290053
0.1383263765874021
0.13386154470849704
0.09103078788850277
0.13817730668693648
0.07248858225918475
0.07086674815657756
0.1046995335411113
0.12169440559505718
0.08931431329783507
0.118368129

0.11306830294866695
0.07554601342730555
0.10334873028479938
0.08065697845675114
0.11146340453181613
0.11430237017939546
0.11208598465087347
0.06257147357051707
0.10226764435739603
0.11597230453322617
0.10390768979736167
0.08262669200818423
0.1157984654495303
0.08474066580076492
0.05781938317803786
0.10525505144196873
0.10129806652010037
0.08129454280533915
0.10410256962053774
0.10095655832892683
0.1091204777301468
0.08383782162059072
0.14331149561028753
0.06308018185019784
0.10786939313482363
0.06985842813853077
0.11631255253981462
0.09087129461144955
0.11416013132857913
0.1087929048270057
0.09585654681087377
0.10327372031090326
0.09438180226782569
0.10515415321605251
0.11405904006977717
0.0774190394984727
0.06985955842010973
0.10876116382455059
0.09884824126113774
0.09085110984789461
0.10663183077298746
0.09824628694240758
0.06476812168957091
0.10863496091154515
0.08003650436188654
0.06069071928505048
0.10613729377298992
0.0965243343419836
0.10848104676133874
0.09516026317276113
0.111

...

0.052323240480597825
0.0567512815984959
0.14240524398007226
0.056030169416487564
0.11199504065296803
0.08460997204960177
0.08237913748201493
0.0857257741014914
0.07411144178687702
0.054004296960930946
0.13107951723824385
0.05346001700302999
0.1018681093271122
0.05541369200089907
0.061634295411687426
0.12347121343266507
0.09481113833529864
0.08300611662708604
0.11657386838963978
0.05235682564099957
0.06849850276559276
0.10490701162473437
0.09787790797280708
0.055262976878367835
0.05235682564099957
0.07737407044786344
0.056912399881436704
0.07767754095828361
0.08461572813453735
0.0569440139807575
0.07566166182369563
0.13697549129152978
0.09814250654334866
0.07986122775567273
0.11000524613916246
0.06989538111589959
0.0819181342720676
0.1009955465264738
0.05692343762067535
0.08254174061523034
0.0763653119207507
0.06089592413542255
0.06969193033393135
0.10112081751948146
0.06444001381951306
0.1266026813563647
0.1321368766960781
0.07139233995143425
0.09760882488864439
0.13363895948572468
0.1

0.08059933838579364
0.10047861935694077
0.1316488060932589
0.10908762854377375
0.053778109875964236
0.10407398535142688
0.06422815443603205
0.07264367085656505
0.053292771794864274
0.058674818539804385
0.12554613021060954
0.13246818606840557
0.10448570815772525
0.05861832302639012
0.09186107157887974
0.06509825877522603
0.05502981173551717
0.06761525945244902
0.07702828135443324
0.12777306981882944
0.07212514385158618
0.13612066514086735
0.053145993975007765
0.05617442597247102
0.08374827481402906
0.0647021502997023
0.061608948432078495
0.11566970617570865
0.10919632011865216
0.07644491162613089
0.08232166847283283
0.05938529815960214
0.11523853661819859
0.06500195747408291
0.08985366159194774
0.11058184721808367
0.05427309327783229
0.07833929225831225
0.08664006939670522
0.08462166080864787
0.07748606983051595
0.0573181051317755
0.06367705600798824
0.08295322838320077
0.08197161608430857
0.06217112079620902
0.053304361771639164
0.10680074086742936
0.1002912706796645
0.0522641539643431

0.05330557696510674
0.13253950386645916
0.05990452486369545
0.08256474623498394
0.062071627108635494
0.12807697645863247
0.05718121692378074
0.05793747350709976
0.05387695545307726
0.100702750801183
0.11998504907386821
0.07040766298985736
0.05604325848269658
0.058028632024530624
0.05230516695516597
0.05626978284275393
0.10079411686898171
0.08021492158791856
0.06441590603625665
0.05496238598307467
0.10337175200028706
0.06705855198617244
0.10390343293356763
0.09588149411008687
0.05334517456655141
0.09933984706399206
0.13332516132490435
0.0846813493060822
0.0641819430511909
0.08902765692211825
0.10824186369788286
0.05347421802973436
0.08282474798101479
0.06200959734779131
0.12581613487427695
0.05701307302178437
0.13550787350184276
0.09944277336206145
0.11242600973975136
0.13316751488975323
0.05846814754920469
0.0793186375464157
0.07162231231814076
0.05624992159487084
0.08335066782779837
0.05338211170120744
0.05230516695516597
0.05662713202336176
0.05983471415461928
0.13986932820306477
0.0

0.06485275866902723
0.10606712059117299
0.05226407435280583
0.053046482060397665
0.058223936289947306
0.10825204702404699
0.05226569061701709
0.053433894406949345
0.05816432467499861
0.052264153964343196
0.05228739060564645
0.12807731290426644
0.052299557631032
0.05228621984087921
0.10942863033462077
0.05329969759766591
0.118372918000791
0.06465689003233503
0.05227966153064918
0.10269547728388762
0.05225896994493244
0.0877760353678613
0.10720684734651728
0.054662320665531984
0.05226440025372573
0.05226267039122472
0.05226886992565721
0.10824394331536653
0.05225899335667753
0.09887681371471023
0.09642929671702097
0.10229662347148155
0.05329517322938187
0.05230511311112771
0.10920299849480206
0.05226616022467605
0.052264153964343196
0.05339512323444004
0.12808451655471975
0.05234994394829149
0.07861119531645142
0.10132759967288402
0.05228571250908329
0.0699285586570565
0.060477374517707405
0.10619775853061172
0.05226049048432936
0.07696162738986888
0.05226221886808499
0.11213528724266375

0.05225901547424454
0.0522606623029473
0.05342827450875019
0.052260342052930446
0.052260406148134805
0.05229062469575664
0.05226501788166369
0.05226267039122472
0.052288716339255935
0.05225900551830559
0.0522593137914953
0.0522590340510737
0.05226352211401265
0.05226407971142444
0.052263898424880724
0.05227696409571968
0.05225890776066735
0.05226381204073083
0.052264153964343196
0.05225896994493244
0.05227021804662576
0.05226907064668856
0.05225890480405821
0.06007402062527785
0.05225969275848109
0.05226672595355955
0.052273021362995144
0.05227004887142036
0.05226611634179132
0.05226389665272492
0.052297164433118416
0.0522606623029473
0.0775579123697566
0.05226236965908943
0.05225900551830559
0.0522742139581953
0.05227552310251074
0.05225889289896995
0.05226039709300933
0.05227241364769191
0.05226134161095847
0.05225887235322803
0.05225933069610727
0.052261170627175044
0.05226043273147895
0.05226271689694429
0.05226128035558645
0.05228567720118634
0.052261999080009516
0.052264153964343

In [ ]:
forward_params['model'].get_optimization_option("val", vectorize=True)

In [ ]:
inversed_model

In [ ]:
plot_1D_err(forward_params, 100, vline_x=inversed_model)

In [ ]:
plot_2D_err(forward_params, [5, 5], 20, points=[[inversed_model[0]], [inversed_model[1]]])

In [ ]:
res = forward_with_trace_calcing(forward_params['model'], forward_params['x_rec'], 
                           dt=forward_params['dt'], 
                           trace_len=forward_params['trace_len'], 
                           wavetypes=forward_params['wavetypes'],
            display_stat=True, visualize_res=False,
                               visualize_seismograms=True
            )

In [ ]:
inversed_model